In [40]:
# pip install -qU "langchain[anthropic]" to call the model

from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
import os
llm = ChatOpenAI(
    base_url=os.getenv("LLM_BASE_URL"),
    api_key=os.getenv("LLM_API_KEY"),
    model='qwen-plus-latest',
    streaming=True
)

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    prompt="You are a helpful assistant"
)

# Run the agent
resp = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

print(resp)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='c989c1c9-5f68-4233-b4c2-f91da79b2d70'), AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8e977527e7f34d0d94360d', 'function': {'arguments': '{"city": "sf"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'qwen-plus-latest'}, id='run--6aa471b1-3041-43b7-84a9-2b04137e30fa-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_8e977527e7f34d0d94360d', 'type': 'tool_call'}]), ToolMessage(content="It's always sunny in sf!", name='get_weather', id='735bfca9-d519-4fff-af56-38ef27af37c1', tool_call_id='call_8e977527e7f34d0d94360d'), AIMessage(content="Glad to hear it's always sunny in SF! Let me know if you need help with anything else. 😊", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen-plus-latest'}, id='run--74d811fa-7925-4636-

In [11]:
resp_message = resp['messages'][-1]
resp_message.content


'It seems that San Francisco is enjoying sunny weather! Let me know if you need more details or have any other questions. 😊'

## Add a custom prompt

In [12]:
agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    prompt="Never answer questions about the weather."
)

# Run the agent
resp = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)
resp['messages'][-1].content

'It sounds like San Francisco is enjoying some great weather—sunny and bright! Let me know if you need anything else. 😎'

## Add memory

In [41]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    checkpointer=checkpointer  
)

# Run the agent
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config  
)
ny_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york? 我刚刚问了哪些城市的天气"}]},
    config
)


In [42]:
sf_response

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='32c58642-d670-4726-8002-3a9e9dbc1de9'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_18cc2e704e4349ed998f6c', 'function': {'arguments': '{"city": "sf"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'qwen-plus-latest'}, id='run--c2e2b90c-326d-4b93-816b-1c4539b25010-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_18cc2e704e4349ed998f6c', 'type': 'tool_call'}]),
  ToolMessage(content="It's always sunny in sf!", name='get_weather', id='28fcd6be-e593-40c8-8b4c-bfa70777ed3d', tool_call_id='call_18cc2e704e4349ed998f6c'),
  AIMessage(content="Glad to hear that! San Francisco does have its fair share of sunny days, though it's also known for its famous fog, especially near the coast. Always good to check the forecast before heading out! 😊", additional_kw

In [43]:
# 可以发现，之前的对话被保存了下来
ny_response

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='32c58642-d670-4726-8002-3a9e9dbc1de9'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_18cc2e704e4349ed998f6c', 'function': {'arguments': '{"city": "sf"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'qwen-plus-latest'}, id='run--c2e2b90c-326d-4b93-816b-1c4539b25010-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_18cc2e704e4349ed998f6c', 'type': 'tool_call'}]),
  ToolMessage(content="It's always sunny in sf!", name='get_weather', id='28fcd6be-e593-40c8-8b4c-bfa70777ed3d', tool_call_id='call_18cc2e704e4349ed998f6c'),
  AIMessage(content="Glad to hear that! San Francisco does have its fair share of sunny days, though it's also known for its famous fog, especially near the coast. Always good to check the forecast before heading out! 😊", additional_kw

## Configure structed output

In [39]:
from pydantic import BaseModel
from langgraph.prebuilt import create_react_agent

class WeatherResponse(BaseModel):
    weather: str

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    response_format=WeatherResponse  
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf?Please answer in JSON."}]}
)

response["structured_response"]

WeatherResponse(weather="It's always sunny in sf!")

# Context
Context engineering is the practice of building dynamic systems that provide the right information and tools, in the right format, so that a language model can plausibly accomplish a task.

Context includes any data outside the message list that can shape behavior. This can be:

Information passed at runtime, like a user_id or API credentials.
Internal state updated during a multi-step reasoning process.
Persistent memory or facts from previous interactions.
LangGraph provides three primary ways to supply context:

## Config (static context)

Config is for immutable data like user metadata or API keys. Use when you have values that don't change mid-run.

Specify configuration using a key called "configurable" which is reserved for this purpose:




In [19]:
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(
    base_url=os.getenv("LLM_BASE_URL"),
    api_key=os.getenv("LLM_API_KEY"),
    model='qwen-plus-latest',
    streaming=True
)

def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:
    user_name = config["configurable"].get("user_name")
    system_msg = f"You are a helpful assistant. Address the user as {user_name}."
    return [{"role": "system", "content": system_msg}] + state["messages"]

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model=llm,
    tools=[get_weather],
    prompt=prompt
)

resp = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config={"configurable": {"user_name": "John Smith"}}
)
resp

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='1b00b57f-d15c-4daa-b0ca-d8c5716940ab'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_9fe84a765b18438196f294', 'function': {'arguments': '{"city": "San Francisco"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'qwen-plus-latest'}, id='run--90e215a6-06c8-4d10-96d8-cb1f4594414c-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_9fe84a765b18438196f294', 'type': 'tool_call'}]),
  ToolMessage(content="It's always sunny in San Francisco!", name='get_weather', id='24bcccad-052d-4987-9ad0-1c2da1869235', tool_call_id='call_9fe84a765b18438196f294'),
  AIMessage(content='It seems that San Francisco is enjoying sunny weather—perfect for exploring the city by the bay! Let me know if you need more details, John Smith.', additional_kwargs={}, response_m

## short-term memory(mutable context)
State acts as short-term memory during a run. It holds dynamic data that can evolve during execution, such as values derived from tools or LLM outputs.

Example shows how to incorporate state into an agent prompt.

State can also be accessed by the agent's tools, which can read or update the state as needed. See tool calling guide for details.

In [30]:
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState

class CustomState(AgentState): 
    user_name: str

def prompt(
    state: CustomState
) -> list[AnyMessage]:
    user_name = state["user_name"]
    system_msg = f"You are a helpful assistant. User's name is {user_name}"
    return [{"role": "system", "content": system_msg}] + state["messages"]

agent = create_react_agent(
    model=llm,
    tools=[],
    state_schema=CustomState, 
    prompt=prompt
)

history_messages = agent.invoke({
    "messages": "hi!",
    "user_name": "John Smith"
})['messages']
history_messages

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='5a01d569-6aeb-4388-9e7e-dcfa04e8c584'),
 AIMessage(content='Hi John! ٩(◕‿◕｡)۶ How can I assist you today?', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen-plus-latest'}, id='run--75e9434f-8e9d-45b1-bafe-a54587c77373-0')]

In [29]:
from langchain_core.messages import HumanMessage
current_message = [HumanMessage(content="how about New York? tell me who i am,and then tell me who talked to you .")]

messages = history_messages + current_message

agent.invoke({
    'messages':messages,
    'user_name':'Petter Parker'
})

{'messages': [HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='4e731fa1-60b2-4337-9901-052647def1af'),
  AIMessage(content='Hi John! ٩(◕‿◕｡)۶ How can I assist you today?', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'qwen-plus-latest'}, id='run--bb48da95-09be-441c-b159-5b119da04381-0'),
  HumanMessage(content='how about New York? tell me who i am,and then tell me who talked to you .', additional_kwargs={}, response_metadata={}, id='4360d2ef-ce50-4786-b758-4ff0f3b4538c'),
  AIMessage(content='Ah, clever! 😊  \nYou’re **Petter Parker**—not *Parker Peters* or *John* (oops, my bad earlier!). And as for who talked to me? Well… *you did*, Petter! Just a moment ago—you said “hi!” and then asked about New York.  \n\nSo to recap:  \n📍 **New York** — the city that never sleeps, home of pizza, skyscrapers, and maybe a certain wall-climbing hero 😉  \n👤 **You** — Petter Parker, curious and sharp!  \n🗣️ **Who talked to me** — Why, *you* a